### 신경망 정의하기

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16*6*6, 120)   # 6x6 from iamge dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # (B, 1, 32, 32)
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        # (B, 6, 15, 15)
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), (2,2))
        # (B, 16, 6, 6)
        x = x.view(-1, self.num_flat_features(x))
        # (B, 16*6*6)
        x = F.relu(self.fc1(x))
        # (B, 120)
        x = F.relu(self.fc2(x))
        # (B, 84)
        x = self.fc3(x)
        # (B, 10)
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]     # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [2]:
net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) # conv1's .weight

10
torch.Size([6, 1, 3, 3])


In [5]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.1097,  0.0318, -0.1109, -0.0298, -0.0020,  0.0077,  0.0568, -0.0282,
         -0.0757,  0.0598]], grad_fn=<AddmmBackward>)


`torch.nn`은 미니-배치(mini-batch)만 지원합니다. `torch.nn` 패키지 전체는 하나의 샘플이 아닌, 샘플들의 미니-배치만을 입력으로 받습니다.<br>
예를 들어, `nnConv2D`는 nSamples x nChannels x Height x Width 의 4차원 Tensor를 입력으로 합니다.<br>
만약 하나의 샘플만 있다면, `input.unsqueeze(0)`을 사용해서 가짜 차원을 추가합니다.

In [6]:
net.zero_grad()
out.backward(torch.randn(1, 10))

### 손실 함수(Loss Function)

In [7]:
output = net(input)
target = torch.randn(10)    # a dummy target, for example
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.3502, grad_fn=<MseLossBackward>)


In [8]:
print(loss.grad_fn)     # MSELoss
print(loss.grad_fn.next_functions[0][0])    # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])   # ReLU

### 역전파(Backprop)

In [10]:
net.zero_grad()     # zeros the gradient buffers of all parameters

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([-0.0014,  0.0033,  0.0033,  0.0039,  0.0094, -0.0059])


### 가중치 갱신

In [11]:
learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [12]:
import torch.optim as optim

# Optimizer를 생성합니다.
optimizer = optim.SGD(net.parameters(), lr=0.01)

# 학습 과정(training loop)에서는 다음과 같습니다:
optimizer.zero_grad()   # zero the gradienet buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()        # Dose the update